In [ ]:
# Connect gdrive
from google.colab import drive 
drive.mount('/content/gdrive')

In [ ]:
# Setting tensorflow and pytorch for concrete dropout
# need to be done first
%tensorflow_version 1.x

In [ ]:
import tensorflow
print(tensorflow.__version__)

In [ ]:
!pip install q keras==2.2.4

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import keras

In [4]:
import models

In [6]:
Mdir = "/content/gdrive/My Drive/Colab Notebooks/"

In [8]:
# import data
Xdir = 'X.npy'
X = np.load(Mdir + Xdir)
dataDir = "Datasets/YawDD dataset/pics.csv"
data = pd.read_csv(Mdir + dataDir)

In [11]:
num_classes = 3
y = data['class']
y = keras.utils.to_categorical(y, num_classes)

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.8, stratify = y)

In [ ]:
# padding for RNN when using complete video file dataset
from keras.preprocessing import sequence
seq_len = 46
vidPad=sequence.pad_sequences(X, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [12]:
# Clear memory
X = None

In [ ]:
np.random.seed(2728)
tensorflow.set_random_seed(58294)

In [ ]:
# inits for concrete layers
N = len(x_train)
wd = 1e-2 / N
dd = 2. / N

In [16]:
# init for single frame
input_shape = (224, 224, 3)
batch_size = 128

In [13]:
# init for RNN
batch_size = 28
input_shape = (46, 128, 128, 3)

In [ ]:
# init for multi frame
batch_size = 128
input_shape = (128, 768, 3)

In [ ]:
modlenet = models.lenet(input_shape, num_classes)

In [ ]:
modlenet = models.lenetDrp(input_shape, num_classes)

In [18]:
modlenet = models.lenetMcDrp(input_shape, num_classes)

In [ ]:
modlenet = models.lenetCncDrp(input_shape, num_classes)

In [ ]:
modLstm = models.lenetLSTM(input_shape, num_classes)

In [ ]:
h1 = modlenet.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=30,
              verbose=1,
              validation_data=(x_test, y_test))

In [ ]:
# stochstic predictions for BNNs
import tqdm

mc_predictions = []
for i in tqdm.tqdm(range(500)):
    y_p = modlenet.predict(X_test, batch_size=128)
    mc_predictions.append(y_p)

In [ ]:
# save predictions for analysis
mcPred = np.asarray(mc_predictions)
Pdir = "mcPredExp1.npy"
np.save(Mdir + Pdir, mcPred)

In [ ]:
# mean accuracy scores
from sklearn.metrics import accuracy_score
mc_ensemble_pred = mcPred.mean(axis=0).argmax(axis=1)
ensemble_acc = accuracy_score(y_test.argmax(axis=1), mc_ensemble_pred)
print("MC-ensemble accuracy: {:.1%}".format(ensemble_acc))